In [2]:
from math import log
import numpy as np
import operator
import matplotlib.pyplot as plt

In [3]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 侮辱性文字, 0 正常言论
    return postingList,classVec

In [4]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

In [6]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

In [29]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [36]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)      #change to ones() 
    p0Denom = 0.0; 
    p1Denom = 0.0                  #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = (p1Num/p1Denom)          #change to log()
    p0Vect = (p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [37]:
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
trainNB0(trainMat, listClasses)

(array([0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.125     ,
        0.04166667, 0.        , 0.        , 0.04166667, 0.        ,
        0.        , 0.04166667, 0.        , 0.08333333, 0.        ,
        0.        , 0.04166667, 0.04166667, 0.04166667, 0.        ,
        0.        , 0.04166667, 0.        , 0.04166667, 0.04166667,
        0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.04166667,
        0.04166667, 0.        ]),
 array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.05263158, 0.05263158, 0.        , 0.05263158,
        0.05263158, 0.        , 0.05263158, 0.05263158, 0.15789474,
        0.05263158, 0.        , 0.10526316, 0.        , 0.05263158,
        0.05263158, 0.05263158, 0.05263158, 0.        , 0.        ,
        0.05263158, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.10526316]),
 0.5)

In [52]:
def trainNB1(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)      #change to ones() 
    p0Denom = 2.0; 
    p1Denom = 2.0                  #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)          #change to log()
    p0Vect = np.log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [53]:
p0V, p1V, pAb= trainNB1(trainMat, listClasses)

In [54]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

In [61]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB1(np.array(trainMat),np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

In [62]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


In [63]:
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [126]:
def textParse(bigString):    #input is big string, #output is word list
    #print ('The string is: ',bigString)
    import re
    listOfTokens = re.split(r'\W+', bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] 

In [164]:
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        #print(i, end='')
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = list(range(50));
    testSet=[]           #create test set
    for i in range(10):
        randIndex = int(np.random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
        
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
        
    p0V,p1V,pSpam = trainNB1(np.array(trainMat),np.array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            #print ("classification error",docList[docIndex])
    print ('Test count %d error count %d rate is: %.2f'%(len(testSet), errorCount,float(errorCount)/len(testSet) * 100))

In [165]:
for i in range(25):
    spamTest()

Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 2 rate is: 20.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 0 rate is: 0.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error count 1 rate is: 10.00
Test count 10 error cou